# Práctica 2 - Detección de anomalías

__Curso__: Statistical Learning II

__Catedrático__: Ing. Luis Leal

__Estudiante__: Dany Rafael Díaz Lux (21000864)

In [1]:
# Importar librerías que utilizaremos
from scipy.stats import multivariate_normal
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

## Cargar y escalar información de entrenamiento

In [2]:
# Cargar información de estaturas
estaturasDf = pd.read_excel('estaturas.xlsx', sheet_name='normales')
# Estandarizar información
estaturasScaler = StandardScaler()
estaturasEstandarizadas = estaturasScaler.fit_transform(estaturasDf)
# Mostrar primeros datos estandarizados
print(estaturasEstandarizadas[0:5, :])

[[ 0.61336991 -0.60120511]
 [ 0.207922    0.16881129]
 [-0.0623766  -0.90921166]
 [ 0.74851921  0.63082112]
 [-1.00842171  0.32281456]]


## Funciones para determinar densidad de probabilidad para datos

In [3]:
# Función para determinar las medias y varianzas de los datos
def obtenerMediasyMatrizCovarianza(X):
    if(type(X).__module__ != np.__name__):
        X = np.array(X)
    
    media = np.mean(X, axis=0)
    matrizCovarianza = np.cov(X.T)
    return media, matrizCovarianza

# Función que calculará para cada observación (con todos sus características) su probabilidad gaussiana conjunta.
def obtenerProbabilidadesGaussianas(X, media, matrizCovarianza):
    modeloProbabilidad = multivariate_normal(mean=media, cov=matrizCovarianza)
    return modeloProbabilidad.pdf(X)

# Función que seleccionará mejor límite de probabilidad para detección de anomalías, 
# tomando como criterio f1-score e información con etiquetas (1=anomalía, 0=normal).
def obtenerProbabilidadLimiteParaAnomalias(probabilidades, etiquetas, divisionRangoProbabilidades = 1000):
    mejor_epsilon = 0
    mejor_f1 = 0
    f1 = 0
    paso = (max(probabilidades) - min(probabilidades)) / divisionRangoProbabilidades
    epsilons = np.arange(min(probabilidades), max(probabilidades), paso)
    for epsilon in list(epsilons):
        predicciones = (probabilidades < epsilon) 
        f1 = f1_score(etiquetas, predicciones, average='binary')
        if f1 > mejor_f1:
            mejor_f1 = f1
            mejor_epsilon = epsilon
    
    return mejor_f1, mejor_epsilon

# predecir si los datos son anómalos o no.
def predecirDatoAnomalo(X, medias, covarianza, epsilon=0.001):
    probabilidades = obtenerProbabilidadesGaussianas(X, medias, covarianza)
    predicciones = (probabilidades < epsilon)
    return predicciones.astype(int)

## Cargar datos normales y anómalos

In [4]:
# Cargar datos "normales", estandarizarlos y etiquetarlos (y=0)
estaturasNormales = pd.read_excel('estaturas.xlsx', sheet_name='valtest(normales)')
estaturasNormalesEstandarizadas = estaturasScaler.transform(estaturasNormales)
estaturasNormalesEstandarizadas = np.c_[estaturasNormalesEstandarizadas, np.zeros(estaturasNormalesEstandarizadas.shape[0])]
# Cargar datos "anómalos", estandarizarlos y etiquetarlos (y=1)
estaturasAnomalas = pd.read_excel('estaturas.xlsx', sheet_name='valtest(anomalias)')
estaturasAnomalasEstandarizadas = estaturasScaler.transform(estaturasAnomalas)
estaturasAnomalasEstandarizadas = np.c_[estaturasAnomalasEstandarizadas, np.ones(estaturasAnomalasEstandarizadas.shape[0])]
# Repartir mitad de datos normales y anómalos en conjuntos de validación y pruebas
random.seed(2022)
indices = random.sample(range(estaturasNormales.shape[0]), estaturasNormales.shape[0] // 2)
normalesValidacion = estaturasNormalesEstandarizadas[indices,:]
normalesPruebas = np.array([list(el) for i, el in enumerate(list(estaturasNormalesEstandarizadas)) if i not in set(indices)])
random.seed(2022)
indices = random.sample(range(estaturasAnomalas.shape[0]), estaturasAnomalas.shape[0] // 2)
anomalosValidacion = estaturasAnomalasEstandarizadas[indices,:]
anomalosPruebas = np.array([list(el) for i, el in enumerate(list(estaturasAnomalasEstandarizadas)) if i not in set(indices)])
estaturasValidacion = np.r_[normalesValidacion, anomalosValidacion]
estaturasPruebas = np.r_[normalesPruebas, anomalosPruebas]
print('Estaturas para validación: ')
print(estaturasValidacion)

Estaturas para validación: 
[[ 1.28911641e+00  1.48080075e-02  0.00000000e+00]
 [-1.68416822e+00  7.84824400e-01  0.00000000e+00]
 [-1.54901892e+00  1.68811286e-01  0.00000000e+00]
 [-1.68416822e+00  1.68811286e-01  0.00000000e+00]
 [-2.12808169e+01  3.38955293e+01  1.00000000e+00]
 [ 2.34315621e+03 -4.22028215e+00  1.00000000e+00]]


## Entrenamiento de modelo, validación

In [5]:
# Obtener medias y matriz de covarianza de datos de entrenamiento
medias, matrizCovarianza = obtenerMediasyMatrizCovarianza(estaturasEstandarizadas)
# Obtener densidades de probabilidad de datos de validación
probabilidades = obtenerProbabilidadesGaussianas(estaturasValidacion[:,:-1], medias, matrizCovarianza)
# Determinar parámetro límite que mejor detecta anomalías
f1, epsilon = obtenerProbabilidadLimiteParaAnomalias(probabilidades, estaturasValidacion[:,-1], 1000)
print('Mejor epsilon encontrado: ', epsilon)
print('Mejor f1_score obtenido: ', f1)

Mejor epsilon encontrado:  6.901402324377259e-05
Mejor f1_score obtenido:  1.0


## Evaluar epsilon en datos de prueba

In [6]:
# Realizar predicciones en datos de prueba
predicciones = predecirDatoAnomalo(estaturasPruebas[:,:-1], medias, matrizCovarianza)
# Evaluar f1-score de predicciones
f1ScorePruebas = f1_score(estaturasPruebas[:,-1], predicciones)
print('f1-score obtenida en datos de prueba:')
print(f1ScorePruebas)

f1-score obtenida en datos de prueba:
1.0
